In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Simple valuation with default recipes

This notebook shows how to value a portfolio using defatul recipes, for an out of the box look at positions and valuations

Attributes
----------
valuation
transactions
recipes
manifests
"""

toggle_code("Hide docstring")

# Simplified Valuation

This notebook illustrates an example of how to use [*GetValuation*](https://www.lusid.com/docs/api/#operation/GetValuation) for a simplified call to the valuation engine that uses a default [*recipe*](https://support.finbourne.com/what-is-a-lusid-recipe-and-how-is-it-used). The default recipe uses a simple `price x quantity` valuation with price source "Lusid".

For an example of a valuation with a customized recipe, see the sample notebook "Valuation with recipe ID".


## Table of contents

- 1. [Load data](#1.-Load-Data)
   * [1.1 Instruments](#1.1-Instruments)
   * [1.2 Portfolio](#1.2-Portfolio)
   * [1.3 Transactions](#1.3-Transactions)
   * [1.4 Quotes](#1.4-Quotes)
- 2. [Run simplified valuations](#2.-Run-simplified-valuations)
    * [2.1 Single-day](#2.1-Single-day)
    * [2.2 Multi-day subtotals](#2.2-Multi-day-subtotals)
    * [2.3 Multi-day ranges](#2.3-Multi-day-ranges)

In [2]:
# Import system packages

# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python

import lusid
import lusid.models as models
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
)
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
import fbnsdkutilities.utilities as utils

import os
import pandas as pd

# Set pandas dataframe display formatting
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = utils.ApiClientFactory(
    lusid,
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

## 1. Load Data

### 1.1 Instruments 

Create a scope and portfolio code, and read the data from the quotes file, containing prices for members of the FTSE100 Index. Here we will simply read the instrument names and identifiers, adding the unique names to LUSID.

In [3]:
scope = "valuation-simplified"
portfolio_code = "EQUITY_UK"

In [4]:
instruments_df = pd.read_excel("data/simple-valuation/ftse-100-prices31-Jul-2020-31-Aug-2020.xlsx")[["name", "figi"]].drop_duplicates()
instruments_df.head()

,name,figi
0,SCOTTISH MORTGAGE INV TR PLC,BBG000BFZMY9
21,FRESNILLO PLC,BBG000VH0TC0
42,AVAST PLC,BBG00KW3SK62
63,POLYMETAL INTERNATIONAL PLC,BBG0025RP8F9
84,AVEVA GROUP PLC,BBG000C21Y87


Create a mapping schema for the instruments using the provided FIGIs as the instrument identifiers. The instruments file is loaded into LUSID. 

In [5]:
instrument_mapping = {
    "identifier_mapping": {
        "Figi": "figi"
    },
    "required": {
        "name": "name"
    },
}

In [6]:
# Instruments can be loaded using a dataframe with file_type set to "instruments"
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=instruments_df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=instrument_mapping["identifier_mapping"],
)

succ, failed, errors = format_instruments_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,101,0,0


The instruments should now be viewable in the [LUSID webtool](https://www.lusid.com/app/home) (*Data Management* >>> *Instruments*)

### 1.2 Portfolio

Create a portfolio in LUSID by setting up a mapping schema which can then be used to load the relevant contents.

In [7]:
portfolio_df = pd.read_excel("data/simple-valuation/portfolio.xlsx")
portfolio_df.head()

,portfolio_code,portfolio_name,portfolio_base_currency,instrument_id,name,txn_id,txn_type,txn_trade_date,txn_settle_date,txn_units,txn_price,txn_consideration,currency,cash_transactions,Strategy,HoldingType,Commission
0,EQUITY_UK,UK Equity Portfolio,GBP,BBG000BWF7M0,ANGLO AMERICAN PLC,T1,Buy,2020-08-03,2020-08-05,105000,"1,910.60","2,016,700.00",GBP,NaN,Tracker,Equity,"5,041.75"
1,EQUITY_UK,UK Equity Portfolio,GBP,BBG000BDCLS8,CRODA INTERNATIONAL PLC,T2,Buy,2020-08-03,2020-08-05,3500,"5,820.00","198,934.00",GBP,NaN,Tracker,Equity,497.33
2,EQUITY_UK,UK Equity Portfolio,GBP,BBG000BD3SC0,ASSOCIATED BRITISH FOODS PLC,T3,Buy,2020-08-03,2020-08-05,51000,"1,786.00","903,500.00",GBP,NaN,Tracker,Equity,"2,258.75"
3,EQUITY_UK,UK Equity Portfolio,GBP,BBG000BD6DG6,BARRATT DEVELOPMENTS PLC,T4,Buy,2020-08-03,2020-08-05,57000,524.40,"303,360.00",GBP,NaN,Tracker,Equity,758.40
4,EQUITY_UK,UK Equity Portfolio,GBP,BBG000NSXQ99,BURBERRY GROUP PLC,T5,Buy,2020-08-03,2020-08-05,93000,"1,280.00","1,190,400.00",GBP,NaN,Tracker,Equity,"2,976.00"


In [8]:
portfolio_mapping = {
    "required": {
        "code": "portfolio_code",
        "display_name": "portfolio_name",
        "base_currency": "$GBP",
    },
    "optional": {"created": "$2020-01-01T00:00:00+00:00"},
}

In [9]:
# A portfolio can be loaded using a dataframe with file_type = "portfolios"
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=portfolio_df,
    mapping_required=portfolio_mapping["required"],
    mapping_optional=portfolio_mapping["optional"],
    file_type="portfolios",
    sub_holding_keys=[],
)

succ, failed = format_portfolios_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,1,0,0


### 1.3 Transactions

Create a transaction mapping schema that uses the provided FIGI identifiers to load the data into LUSID.

In [10]:
transaction_mapping = {
    "identifier_mapping": {
        "Figi": "instrument_id",
    },
    "required": {
        "code": "portfolio_code",
        "transaction_id": "txn_id",
        "type": "txn_type",
        "transaction_price.price": "txn_price",
        "transaction_price.type": "$Price",
        "total_consideration.amount": "txn_consideration",
        "units": "txn_units",
        "transaction_date": "txn_trade_date",
        "total_consideration.currency": "portfolio_base_currency",
        "settlement_date": "txn_settle_date",
    },
    "optional": {},
    "properties": [],
}

In [11]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=portfolio_df,
    mapping_required=transaction_mapping["required"],
    mapping_optional=transaction_mapping["optional"],
    file_type="transactions",
    identifier_mapping=transaction_mapping["identifier_mapping"],
    property_columns=transaction_mapping["properties"],
    properties_scope=scope,
)
    
succ, failed = format_transactions_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,1,0,0


### 1.4 Quotes

Load the source quotes data which containins open and close prices for members of the FTSE100 Index between July 31st and August 31st. This is the pricing data that will be used later in valuation.

In [12]:
quotes_df = pd.read_excel("data/simple-valuation/ftse-100-prices31-Jul-2020-31-Aug-2020.xlsx")
quotes_df.head()

,date,ticker,name,figi,Sector,open_price,close_price
0,2020-08-28T00:00:00.000Z,SMT.L,SCOTTISH MORTGAGE INV TR PLC,BBG000BFZMY9,Equity Investment Instruments,960.00,961.50
1,2020-08-27T00:00:00.000Z,SMT.L,SCOTTISH MORTGAGE INV TR PLC,BBG000BFZMY9,Equity Investment Instruments,958.00,964.50
2,2020-08-26T00:00:00.000Z,SMT.L,SCOTTISH MORTGAGE INV TR PLC,BBG000BFZMY9,Equity Investment Instruments,946.50,953.00
3,2020-08-25T00:00:00.000Z,SMT.L,SCOTTISH MORTGAGE INV TR PLC,BBG000BFZMY9,Equity Investment Instruments,942.00,938.00
4,2020-08-24T00:00:00.000Z,SMT.L,SCOTTISH MORTGAGE INV TR PLC,BBG000BFZMY9,Equity Investment Instruments,935.50,936.00


The current quotes source data uses FIGI as the core unique identifier, but we can call the API and create unique LUSID identifiers (LUID). These will later be used in our valuation call by mapping them against the previously set transactions.

In [13]:
def add_luid_id(data_frame):
    client_ids = pd.DataFrame(list(data_frame["figi"].unique()), columns=["figi"])

    # Call lusid_instrument_id to the API for creating the LUIDs   
    client_ids["LUID"] = client_ids["figi"].apply(
    lambda x: api_factory.build(lusid.api.InstrumentsApi).get_instrument(
        identifier_type="Figi",
        identifier=x).lusid_instrument_id)
    client_ids = client_ids.set_index("figi")
    data_frame['LUID'] = data_frame["figi"].apply(lambda x: client_ids.loc[x]["LUID"])
    return data_frame

df = add_luid_id(quotes_df)

# Check the first one to see that LUID was added
df[["name", "LUID"]].head(1)

,name,LUID
0,SCOTTISH MORTGAGE INV TR PLC,LUID_ZELLWKTX


Create a mapping schema for the for the quotes dataframe to read the using the LUIDs.

In [14]:
quotes_mapping = {
    "quote_id.quote_series_id.instrument_id_type": "$LusidInstrumentId",
    "quote_id.effective_at": "date",
    "quote_id.quote_series_id.provider": "$Lusid",
    "quote_id.quote_series_id.quote_type": "$Price",
    "quote_id.quote_series_id.instrument_id": "LUID",
    "metric_value.unit": "$GBP",
}

We can use the end of day close prices for mapping the quotes source data (pricing data assumed to be using "mid" quotes).

In [15]:
quotes_mapping["quote_id.quote_series_id.field"] ="$mid"
quotes_mapping["metric_value.value"] = "close_price"

result = load_from_data_frame(
    api_factory = api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=quotes_mapping,
    mapping_optional={},
    file_type="quotes"
)

succ, failed, errors = format_quotes_response(result)
display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]))

,success,failed,errors
0,2121,0,0


## 2. Run simplified valuation

### 2.1 Single-day

Perform a valuation on the portfolio by using the simple valuation call from LUSID. Notice we pass "default" for the recipe code when using *ValuationRequest*.

- Recipe default attributes:

| Price source/supplier | Instrument ID | Quote Type | Pricing Model | Calculation     |
| :--------------------:| :----------:  | :---------:|:-------------:|:-----------:    |
| LUSID                 | LUID          | Price (mid)| Simple Static | Quanity x price |


In [16]:
# Setup the aggregation request 
def aggregation_request(effectiveAt):
    return models.ValuationRequest( 
        recipe_id = models.ResourceId(
            scope = scope,
            code = "default"
        ),
        metrics = [
            models.AggregateSpec("Instrument/default/Name", "Value"),
            models.AggregateSpec("Valuation/PV", "Proportion"),
            models.AggregateSpec("Valuation/PV", "Sum"),
            models.AggregateSpec("Holding/default/Units", "Sum"),
        ],
        group_by=["Instrument/default/Name"],
        # choose the valuation date for the request - set using effectiveAt
        valuation_schedule=models.ValuationSchedule(effective_at=effectiveAt),
        portfolio_entity_ids = [models.PortfolioEntityId(
                                                        scope = scope,
                                                        code = portfolio_code,
                                                        portfolio_entity_type="SinglePortfolio" 
            )]
        )

    

In [17]:
# Pull the data aggregation by passing the effectiveAt date
aggregation_api = api_factory.build(lusid.AggregationApi)
aggregation = aggregation_api.get_valuation(
                                            valuation_request=aggregation_request("2020-08-24T01:01:00.000Z")
            )
pd.DataFrame(aggregation.data)

,Instrument/default/Name,Proportion(Valuation/PV),Sum(Valuation/PV),Sum(Holding/default/Units)
0,ANGLO AMERICAN PLC,0.10,"200,172,000.00","105,000.00"
1,CRODA INTERNATIONAL PLC,0.01,"20,965,000.00","3,500.00"
2,ASSOCIATED BRITISH FOODS PLC,0.05,"104,193,000.00","51,000.00"
3,BARRATT DEVELOPMENTS PLC,0.02,"29,982,000.00","57,000.00"
4,BURBERRY GROUP PLC,0.10,"200,700,500.00","143,000.00"
5,UNILEVER PLC,0.07,"137,430,000.00","30,000.00"
6,PEARSON PLC,0.09,"176,386,000.00","301,000.00"
7,TESCO PLC,0.02,"30,191,000.00","133,000.00"
8,WHITBREAD PLC,0.04,"85,320,000.00","36,000.00"
9,LLOYDS BANKING GROUP PLC,0.09,"164,450,000.00","5,750,000.00"


### 2.2 Multi-day subtotals

Using the same valuation request, we are also able to inspect the evolution of the portfolio holdings and their value for a custom date range. Using a function that groups by <code>"Analytic/default/ValuationDate"</code> the function can call LUSID for an overall PV of the portfolio as a time series.   

In [18]:
def aggregation_interval_request(effectiveFrom, effectiveAt):
    return models.ValuationRequest( 
        recipe_id = models.ResourceId(
            scope = scope,
            code = "default"
        ),
        metrics = [
            models.AggregateSpec("Analytic/default/ValuationDate", "Value"),
            models.AggregateSpec("Valuation/PvInReportCcy", "Sum"),
        ],
        group_by=["Analytic/default/ValuationDate"],
        # choose the valuation interval for the request - set using effectiveFrom and effectiveAt
        valuation_schedule=models.ValuationSchedule(effective_from = effectiveFrom, effective_at=effectiveAt),
        portfolio_entity_ids = [models.PortfolioEntityId(
                                                        scope = scope,
                                                        code = portfolio_code,
                                                        portfolio_entity_type="SinglePortfolio" 
            )]
        )

In [19]:
aggregation_api = api_factory.build(lusid.AggregationApi)
aggregation = aggregation_api.get_valuation(
                                            valuation_request=aggregation_interval_request(
                                                "2020-08-24T01:01:00.000Z", 
                                                "2020-08-28T01:01:00.000Z")
            )
pd.DataFrame(aggregation.data)

,Analytic/default/ValuationDate,Sum(Valuation/PvInReportCcy)
0,2020-08-24T01:01:00.0000000+00:00,"1,915,129,144.00"
1,2020-08-25T01:01:00.0000000+00:00,"1,895,571,644.00"
2,2020-08-26T01:01:00.0000000+00:00,"1,904,319,144.00"
3,2020-08-27T01:01:00.0000000+00:00,"1,891,905,444.00"
4,2020-08-28T01:01:00.0000000+00:00,"1,882,259,544.00"


### 2.3 Multi-day ranges

Given the new function now holds data across the selected period, we can also apply other types of specifications in the aggregation metrics. For example, we can see the min/max range for the valuation of each holding in the selected time period. This can illustrate how a stock's volatility can drift the exposure of the portfolio, which can be notable for longer periods. 

In [20]:
def aggregation_interval_request(effectiveFrom, effectiveAt):
    return models.ValuationRequest( 
        recipe_id = models.ResourceId(
            scope = scope,
            code = "default"
        ),
        metrics = [
            models.AggregateSpec("Instrument/default/Name", "Value"),
            models.AggregateSpec("Valuation/PvInReportCcy", "Min"),
            models.AggregateSpec("Valuation/PvInReportCcy", "Max"),
        ],
        group_by=["Instrument/default/Name"],
        # choose the valuation interval for the request - set using effectiveFrom and effectiveAt
        valuation_schedule=models.ValuationSchedule(effective_from = effectiveFrom, effective_at=effectiveAt),
        portfolio_entity_ids = [models.PortfolioEntityId(
                                                        scope = scope,
                                                        code = portfolio_code,
                                                        portfolio_entity_type="SinglePortfolio" 
            )]
        )

aggregation_api = api_factory.build(lusid.AggregationApi)
aggregation = aggregation_api.get_valuation(
                                            valuation_request=aggregation_interval_request(
                                                "2020-08-24T01:01:00.000Z", 
                                                "2020-08-28T01:01:00.000Z")
            )
pd.DataFrame(aggregation.data)

,Instrument/default/Name,Min(Valuation/PvInReportCcy),Max(Valuation/PvInReportCcy)
0,ANGLO AMERICAN PLC,"189,399,000.00","200,172,000.00"
1,CRODA INTERNATIONAL PLC,"20,650,000.00","21,133,000.00"
2,ASSOCIATED BRITISH FOODS PLC,"104,193,000.00","104,958,000.00"
3,BARRATT DEVELOPMENTS PLC,"29,058,600.00","29,982,000.00"
4,BURBERRY GROUP PLC,"200,700,500.00","207,493,000.00"
5,UNILEVER PLC,"133,800,000.00","137,430,000.00"
6,PEARSON PLC,"168,379,400.00","176,386,000.00"
7,TESCO PLC,"29,087,100.00","30,191,000.00"
8,WHITBREAD PLC,"85,320,000.00","91,044,000.00"
9,LLOYDS BANKING GROUP PLC,"162,150,000.00","164,450,000.00"
